# Custom Model

In [79]:
import tensorflow as tf

import keras

from keras import layers
from keras import models
from keras import optimizers

from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

import sklearn as sk




## Dataset Import

In [80]:
cifar10 = tf.keras.datasets.cifar10
(X_train, Y_train), (X_test,Y_test) = cifar10.load_data()


X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)



In [ ]:
cmi = keras.Input(shape=(None,None,3))

end_models = []

filter_counts =  [32, 16, 16, 32, 32, 64, 64]
kernel_sizes  =  [3,  13, 11,  9,  7,  5,  3]

from keras.callbacks import LearningRateScheduler

for i in range(len(filter_counts)):
    #convert X_sample to float32

    #if the model file already exists, skip it
    inner_model = None

    x_model = None

    try:
        inner_model = models.load_model(f'model_{filter_counts[i]}_{kernel_sizes[i]}_eps.keras', compile=False)
        print(f'Loaded model_{filter_counts[i]}_{kernel_sizes[i]}_eps.keras')
        inner_model.trainable = True
    except:

        inp = layers.Input(shape=(32,32,3))

        x1 = layers.Conv2D(filter_counts[i], (kernel_sizes[i], kernel_sizes[i]), activation='relu', padding='same')(inp)
        x1ref = x1
        x  = layers.Dropout(0.25)(x1)
        x  = layers.BatchNormalization()(x)

        x3 = layers.Conv2D(2*filter_counts[i], (3, 3), activation='relu', padding='same')(x)
        x  = layers.Dropout(0.25)(x3)
        x  = layers.BatchNormalization()(x)

        x4 = layers.Conv2D(2*filter_counts[i], (3, 3), activation='relu', padding='same')(x)
        x  = layers.Dropout(0.25)(x4)
        x  = layers.BatchNormalization()(x)
        x  = layers.MaxPooling2D((2,2))(x)

        x5 = layers.Conv2D(4*filter_counts[i], (3, 3), activation='relu', padding='same')(x)
        x  = layers.Dropout(0.25)(x5)
        x  = layers.BatchNormalization()(x)

        x6 = layers.Conv2D(4*filter_counts[i], (3, 3), activation='relu', padding='same')(x)
        x  = layers.Dropout(0.25)(x6)
        x  = layers.BatchNormalization()(x)
        x  = layers.MaxPooling2D((2,2))(x)

        xB = layers.Add()([x3, x4])
        xC = layers.Add()([x5, x6])

        xAf = layers.Flatten()(x1)
        xBf = layers.Flatten()(xB)
        xCf = layers.Flatten()(xC)
        xf  = layers.Flatten()(x)

        x = layers.Concatenate()([xAf, xBf, xCf, xf])

        x_model = models.Model(inputs=inp, outputs=x)


        inner_model = models.Sequential()
        inner_model.add(layers.InputLayer(shape=(32,32,3)))
        inner_model.add(x_model)


    inner_model.add(layers.Dense(1024, activation='relu'))
    inner_model.add(layers.Dense(10, activation='softmax'))

    inner_model.summary()

    print(f"Training model_{filter_counts[i]}_{kernel_sizes[i]}_eps.keras")

    optimizer = optimizers.Adam(learning_rate=0.001)

    inner_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # inner_model.evaluate(X_test, Y_test, verbose=1)

    inner_model.fit(X_train, Y_train, epochs=30, validation_data=(X_test,Y_test), verbose=1)

    inner_model.pop(rebuild=True)
    inner_model.pop(rebuild=True)

    inner_model.save(f'model_{filter_counts[i]}_{kernel_sizes[i]}_eps.keras')



Loaded model_32_3_eps.keras


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ functional_91 (Functional)      │ (None, 139264)         │       279,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 1024)           │   142,607,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 10)             │        10,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,897,034 (545.11 MB)

 Trainable params: 142,896,202 (545.11 MB)

 Non-trainable params: 832 (3.25 KB)

Training model_32_3_eps.keras
Epoch 1/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 124s 77ms/step - accuracy: 0.3605 - loss: 277295.1250 - val_accuracy: 0.4131 - val_loss: 127232.7266
Epoch 2/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 119s 76ms/step - accuracy: 0.4451 - loss: 132765.2656 - val_accuracy: 0.4299 - val_loss: 150767.8750
Epoch 3/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 120s 77ms/step - accuracy: 0.4890 - loss: 126935.5391 - val_accuracy: 0.5407 - val_loss: 136431.8281
Epoch 4/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 120s 77ms/step - accuracy: 0.5109 - loss: 149878.9688 - val_accuracy: 0.5082 - val_loss: 170083.1875
Epoch 5/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 121s 78ms/step - accuracy: 0.5014 - loss: 193934.0312 - val_accuracy: 0.4403 - val_loss: 323586.3750
Epoch 6/30
 909/1563 ━━━━━━━━━━━━━━━━━━━━ 48s 75ms/step - accuracy: 0.5098 - loss: 234428.2656

In [ ]:
#load the cifar100 dataset
cifar100 = tf.keras.datasets.cifar100

(X_train, Y_train), (X_test,Y_test) = cifar100.load_data()

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

print(Y_test.shape)

the_models = []

for i in range(len(filter_counts)):
    inner_model = models.load_model(f'model_{filter_counts[i]}_{kernel_sizes[i]}_eps.keras')
    inner_model.trainable = False

    #Fine tune the model
    model = models.Sequential()
    model.add(layers.InputLayer(shape=(32,32,3)))
    model.add(inner_model)
    model.add(layers.GlobalAveragePooling())
    model.add(layers.Dense(256, activation='tanh'))
    model.add(layers.Dense(100, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, Y_train, epochs=10, batch_size=64)

    model.trainable = False

    the_models.append(model)




(10000, 100)


/Users/thomasjones/.pyenv/versions/finproj/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/20
343/782 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.0687 - loss: 4.3005

In [ ]:

#make the models trainable
for model in the_models:
    model.trainable = False


#Construct an ensemble model
inp = layers.Input(shape=(32,32,3))
x = layers.Add()([model(inp) for model in the_models])
x = layers.Dense(100, activation='softmax')(x)

ensemble_model = models.Model(inp, x)

print(ensemble_model.summary())
print(Y_train.shape)
print(X_train.shape)

ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

ensemble_model.fit(X_train, Y_train, epochs=10, batch_size=64)

#Create a confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import numpy as np

Y_pred = ensemble_model.predict(X_test)

print(Y_pred.shape)
print(Y_test.shape)

yp_max = np.argmax(Y_pred, axis=1)
yt_max = np.argmax(Y_test, axis=1)

#print the metrics
print(classification_report(yt_max, yp_max))

print(confusion_matrix(yt_max, yp_max))


Model: "functional_234"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_6        │ (None, 100)       │  1,172,644 │ input_layer_13[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_7        │ (None, 100)       │  1,170,340 │ input_layer_13[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_8        │ (None, 100)       │  2,429,028 │ input_layer_13[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_9        │ (None, 100)       │  2,425,956 │ input_layer_13[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_10       │ (None, 100)       │  5,354,212 │ input_layer_13[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_11       │ (None, 100)       │  5,351,140 │ input_layer_13[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 100)       │          0 │ sequential_6[0][… │
│                     │                   │            │ sequential_7[0][… │
│                     │                   │            │ sequential_8[0][… │
│                     │                   │            │ sequential_9[0][… │
│                     │                   │            │ sequential_10[0]… │
│                     │                   │            │ sequential_11[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 100)       │     10,100 │ add[0][0]         │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,913,420 (68.33 MB)

 Trainable params: 10,100 (39.45 KB)

 Non-trainable params: 17,903,320 (68.30 MB)

None
(50000, 100)
(50000, 32, 32, 3)
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 27ms/step - accuracy: 0.1340 - loss: 4.2435
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.3669 - loss: 3.2690
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.4278 - loss: 2.7838
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.4426 - loss: 2.5387
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.4566 - loss: 2.3641
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.4655 - loss: 2.2604
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.4751 - loss: 2.1777
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.4768 - loss: 2.1312
Epoch 9/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.4821 - loss: 2.0763
Epoch 10/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.4860 - loss: 2.0517
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step
(10000, 100)
(10000, 100)
              

In [ ]:
confusion_matrix(yt_max, yp_max)

array([[67,  2,  0, ...,  0,  0,  0],
       [ 2, 45,  1, ...,  0,  0,  2],
       [ 3,  1, 25, ...,  0,  9,  0],
       ...,
       [ 0,  0,  1, ..., 34,  0,  1],
       [ 1,  1,  8, ...,  0, 23,  1],
       [ 0,  1,  2, ...,  0,  0, 35]])